In [ ]:
import tensorflow as tf
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

# Time windows
train a model to forecast the next step given the previous 20 steps

---
so lets create a dataset with a 20 step window for training


In [ ]:
dataset = tf.data.Dataset.range(20)
for val in dataset:
  print(val.numpy())

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19


In [ ]:
dataset = tf.data.Dataset.range(20)
dataset = dataset.window(5,shift=1)
for window_dataset in dataset:
  for val in window_dataset:
    print(val.numpy(),end=' ')
  print()

0 1 2 3 4 
1 2 3 4 5 
2 3 4 5 6 
3 4 5 6 7 
4 5 6 7 8 
5 6 7 8 9 
6 7 8 9 10 
7 8 9 10 11 
8 9 10 11 12 
9 10 11 12 13 
10 11 12 13 14 
11 12 13 14 15 
12 13 14 15 16 
13 14 15 16 17 
14 15 16 17 18 
15 16 17 18 19 
16 17 18 19 
17 18 19 
18 19 
19 


In [ ]:
dataset = tf.data.Dataset.range(20)
dataset = dataset.window(5, shift=1, drop_remainder=True)
for window_dataset in dataset:
    for val in window_dataset:
        print(val.numpy(), end=" ")
    print()

0 1 2 3 4 
1 2 3 4 5 
2 3 4 5 6 
3 4 5 6 7 
4 5 6 7 8 
5 6 7 8 9 
6 7 8 9 10 
7 8 9 10 11 
8 9 10 11 12 
9 10 11 12 13 
10 11 12 13 14 
11 12 13 14 15 
12 13 14 15 16 
13 14 15 16 17 
14 15 16 17 18 
15 16 17 18 19 


In [ ]:
dataset = tf.data.Dataset.range(20)
dataset = dataset.window(5,shift=1,drop_remainder=True)
dataset = dataset.flat_map(lambda window: window.batch(5))
for window in dataset:
  print(window.numpy())

[0 1 2 3 4]
[1 2 3 4 5]
[2 3 4 5 6]
[3 4 5 6 7]
[4 5 6 7 8]
[5 6 7 8 9]


In [ ]:
dataset = tf.data.Dataset.range(20)
dataset = dataset.window(5,shift=1,drop_remainder=True)
dataset = dataset.flat_map(lambda window:window.batch(5))
dataset = dataset.map(lambda window : (window[:-1],window[-1:]))

for x,y in dataset:
  print(x.numpy(),y.numpy())

[0 1 2 3] [4]
[1 2 3 4] [5]
[2 3 4 5] [6]
[3 4 5 6] [7]
[4 5 6 7] [8]
[5 6 7 8] [9]
[6 7 8 9] [10]
[ 7  8  9 10] [11]
[ 8  9 10 11] [12]
[ 9 10 11 12] [13]
[10 11 12 13] [14]
[11 12 13 14] [15]
[12 13 14 15] [16]
[13 14 15 16] [17]
[14 15 16 17] [18]
[15 16 17 18] [19]


In [ ]:
dataset = tf.data.Dataset.range(20)
dataset = dataset.window(5,shift=1,drop_remainder=True)
dataset = dataset.flat_map(lambda window:window.batch(5))

dataset = dataset.map(lambda window:(window[:-1],window[-1:]))
dataset = dataset.shuffle(buffer_size=10)

for x,y in dataset:
  print(x.numpy(),y.numpy())

[0 1 2 3] [4]
[ 8  9 10 11] [12]
[4 5 6 7] [8]
[1 2 3 4] [5]
[11 12 13 14] [15]
[14 15 16 17] [18]
[6 7 8 9] [10]
[ 7  8  9 10] [11]
[10 11 12 13] [14]
[5 6 7 8] [9]
[ 9 10 11 12] [13]
[15 16 17 18] [19]
[13 14 15 16] [17]
[12 13 14 15] [16]
[3 4 5 6] [7]
[2 3 4 5] [6]


In [ ]:
dataset = tf.data.Dataset.range(20)
dataset = dataset.window(5, shift=1, drop_remainder=True)
dataset = dataset.flat_map(lambda window: window.batch(5))
dataset = dataset.map(lambda window: (window[:-1], window[-1:]))
dataset = dataset.shuffle(buffer_size=20)
dataset = dataset.batch(2).prefetch(1)
for x,y in dataset:
  print("x = ",x.numpy())
  print("y = ",y.numpy())

x =  [[14 15 16 17]
 [ 5  6  7  8]]
y =  [[18]
 [ 9]]
x =  [[ 6  7  8  9]
 [11 12 13 14]]
y =  [[10]
 [15]]
x =  [[0 1 2 3]
 [4 5 6 7]]
y =  [[4]
 [8]]
x =  [[ 9 10 11 12]
 [ 8  9 10 11]]
y =  [[13]
 [12]]
x =  [[12 13 14 15]
 [13 14 15 16]]
y =  [[16]
 [17]]
x =  [[ 7  8  9 10]
 [ 3  4  5  6]]
y =  [[11]
 [ 7]]
x =  [[2 3 4 5]
 [1 2 3 4]]
y =  [[6]
 [5]]
x =  [[10 11 12 13]
 [15 16 17 18]]
y =  [[14]
 [19]]


# Will make any time series into a tensorflow dataset

In [1]:
def window_dataset(series, window_size, batch_size=32,
                   shuffle_buffer=1000):
    dataset = tf.data.Dataset.from_tensor_slices(series)
    dataset = dataset.window(window_size + 1, shift=1, drop_remainder=True)
    dataset = dataset.flat_map(lambda window: window.batch(window_size + 1))
    dataset = dataset.shuffle(shuffle_buffer)
    dataset = dataset.map(lambda window: (window[:-1], window[-1]))
    dataset = dataset.batch(batch_size).prefetch(1)
    return dataset

# Make a model forecast 

In [2]:
def model_forecast(model,series,window_size):
  ds = tf.data.Dataset.from_tensor_slices(series)
  ds = ds.window(window_size,shift=1,drop_remainder=True)
  ds = ds.flat_map(lambda w: w.batch(window_size))
  ds = ds.batch(32).prefetch(1)
  return model.predict(ds)